# " Impact of COVID-19: Cascading Risks"
> "Sectoral Health and Economic risks"
- toc: true
- math: true
- badges: false

**Sophie Osotimehin and Latchezar Popov**

*Visualizations by Julien Duranton*

In [1]:
#hide_input
import ipywidgets as widgets
import altair as alt
from IPython.display import display
from ipywidgets import interact
import pandas as pd
from vega_datasets import data as data_world
import sys
import os


root = "C:\\Users\\osotimehin_s\\Documents\\GitHub\\blog\\input_data\\covid_input\\"

data_agg = pd.read_csv( root + 'results_socialcons_agg.csv')
data_sec = pd.read_csv(root + "results_socialcons_sec.csv")
mapping_cases = pd.read_csv(root + "results_socialcons_cases.csv")
data_agg_ess = pd.read_csv( root + 'results_essential_agg.csv')
data_sec_ess = pd.read_csv(root + "results_essential_sec.csv")
mapping_industry = pd.read_excel(root + "corres_WIOD_2016_2013.xlsx", sheet_name='matlabcode')
country_codes = pd.read_csv(root + 'countries_codes_and_coordinates.csv')
mapping_country = pd.read_csv(root + "table_coucode.csv")


#format Data Agg
data_agg['Name'] = [mapping_cases[mapping_cases.Var1 == data_agg.casecode[i]].Var2.values[0] 
                         for i in data_agg.index.tolist()]
data_agg['label'] = [mapping_industry[mapping_industry.matlabcode == data_agg.casecode[i]].indname.values[0] if
data_agg.casecode[i] != 0 else 'All Social Consumption' for i in data_agg.index.tolist()]

data_agg['country'] = [mapping_country[mapping_country.coucodelist == data_agg.coucode[i]].coulist.values[0]  
                       for i in data_agg.index.tolist()]
data_agg = data_agg.rename(columns={'dGDP':'GDP','ecorisk_tot':'Total Output'})


#format Data country map
country_codes = country_codes.rename(columns={'Alpha-3 code':'country'})                                                                      
data_countries = pd.DataFrame(data_agg.country.unique(), columns=['country'])
data_countries = data_countries.merge(country_codes, on='country', how='left')


#format Data sec
data_sec = data_sec.rename(columns={'ecorisk':'Output','dCi':'Consumption'})

data_sec['Name'] = [mapping_cases[mapping_cases.Var1 == data_sec.casecode[i]].Var2.values[0] 
                         for i in data_sec.index.tolist()]
data_sec['label'] = [mapping_industry[mapping_industry.matlabcode == data_sec.casecode[i]].indname.values[0] if
data_sec.casecode[i] != 0 else 'All Social Consumption' for i in data_sec.index.tolist()]

data_sec['dsName'] = [mapping_industry[mapping_industry.matlabcode == data_sec.indcode[i]].indname.values[0] 
                         for i in data_sec.index.tolist()]
data_sec['country'] = [mapping_country[mapping_country.coucodelist == data_sec.coucode[i]].coulist.values[0]  
                       for i in data_sec.index.tolist()]

data_sec['type'] = ['closed' if (data_sec.indcode[i] == data_sec.casecode[i]) or 
                    ((data_sec.casecode[i] == 0) & (data_sec.indcode[i] in data_sec.casecode.unique().tolist())) else 
                    'second order' for i in data_sec.index.tolist()]


#format Data Agg
data_agg_ess['country'] = [mapping_country[mapping_country.coucodelist == data_agg_ess.coucode[i]].coulist.values[0]  
                       for i in data_agg_ess.index.tolist()]
data_agg_ess = data_agg_ess.rename(columns={'emp_needed_tot':'Employment', 'tele_tot':'Teleworker_sector','hrisk_tot':'health_risk'})
data_agg_ess['Name']= ['All Sectors' for i in range(data_agg_ess.shape[0])]
data_agg_ess['indcode'] = [0 for i in range(data_agg_ess.shape[0])]
data_agg_ess['essential'] = [2 for i in range(data_agg_ess.shape[0])]



#format Data sec
data_sec_ess['country'] = [mapping_country[mapping_country.coucodelist == data_sec_ess.coucode[i]].coulist.values[0]  
                       for i in data_sec_ess.index.tolist()]
data_sec_ess['Name'] = [mapping_industry[mapping_industry.matlabcode == data_sec_ess.indcode[i]].indname.values[0] 
                         for i in data_sec_ess.index.tolist()]
data_sec_ess['essential'] = [mapping_industry[mapping_industry.matlabcode == data_sec_ess.indcode[i]].essential.values[0]
                            for i in data_sec_ess.index.tolist()]

data_sec_ess = data_sec_ess.rename(columns={'emp_needed':'Employment', 'tele':'Teleworker_sector',
                                            'hrisk':'health_risk'})
data_sec_ess = data_sec_ess.append(data_agg_ess, ignore_index=True)

data_sec_ess['essential_text'] = ['Essential' if data_sec_ess.essential[i] == 1 else 
                             'Non-Essential' if data_sec_ess.essential[i] == 0 else 
                             'All Sectors' for i in data_sec_ess.index.tolist()]

data_sec_ess['color'] = ['blue' if data_sec_ess.essential[i] == 1 else 
                             'lightblue' if data_sec_ess.essential[i] == 0 else 
                             'orange' for i in data_sec_ess.index.tolist()]


data_sec_ess = data_sec_ess.fillna(value=0)

# Add country df
data_country = pd.DataFrame(data_sec_ess.country.unique(),columns=['country'])
data_country['health_risk'] = [data_sec_ess[(data_sec_ess.country == data_country.country[i]) & 
                                            (data_sec_ess.Name == 'All Sectors')].health_risk.values[0] 
                               for i in data_country.index.to_list()]
data_country['economic_risk'] = [data_agg[(data_agg.country == data_country.country[i]) & 
                                            (data_agg.Name == 'close_all')].GDP.values[0] 
                               for i in data_country.index.to_list()]

data_country.loc['mean'] = data_country.mean()
data_country.country['mean'] = 'Mean'

FileNotFoundError: ignored

## Introduction

The covid-19 pandemic poses great public health and economic challenges to countries around the world. Slowing down the spread of the pandemic requires implementing social distancing measures that disrupt economic activity. At the same time, some activities need to be maintained, which puts workers in these sectors at risk of contracting the virus. How does the workers' exposure to these health and economic risks vary across sectors? How do
these risks propagate across sectors?

## Methodology

We focus in this paper on how risk cascade from the two groups of workers the hardest it by the pandemic. 


*   The first group of workers is those working in sectors providing sential goods and services, such as health care, food or pharmaceutical products. These
workers face a higher health risk than other workers: at the risk of being infected, workers in
those sectors are asked to continuing working to ensure that the population's essential needs
are met, even when strict lockdown measures are imposed.
*   The second group of workers is
those working in establishments providing services to the public, such as hotels, restaurants,
or cinemas, which we call social consumption sectors. These workers face a higher economic
risk than other workers. These establishments are likely to operate at low occupancy, even as government-mandated closings are lifted, both because social distancing reduces the number of customers that they
can serve and because customers will most likely avoid these establishments until a vaccine
or a treatment becomes available.

We provide, for 43 countries, a measure of each sectors' exposure to the health and
economic risk that captures the spillover from the two groups of workers. 
The health risk is
the risk coming from meeting the households' essential needs and the economic risk is the risk
due to the unprecedented reduction in the demand for social-consumption establishments.
We use a simple input-output framework, calibrated on data from the World Input Output
Database, to compute the proportion of workers exposed to each type of risk and the spillover
eects. We measure the health risk as the proportion of workers needed at the workplace
to maintain the pre-pandemic household consumption in essential goods and services. We
combine our employment result with the recent work by Dingel and Neiman (2020), which
gives a lower bound for the fraction of each sector's jobs that can be done at home, to derive
the health risk measure. We measure the economic risk as the decline in employment induced
by the collapse of the demand for social consumption.

We describe the methodology in detail in the [working paper](https://).



In [0]:
#hide_input


source = alt.topo_feature(data_world.world_110m.url, 'countries')


alt.data_transformers.disable_max_rows()

background = alt.Chart(source).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=600,
    height=300
).project(type='equirectangular')

selector = alt.selection_single(empty='none', fields=['country'],bind='legend', init={'country':'FRA'})
selector_sector = alt.selection_single(empty='none', fields=['Name'], init={'Name':'close_all'})


# Points and text
hover = alt.selection(type='single', on='mouseover', nearest=True,
                      fields=['Latitude (average)', 'Longitude (average)'])

base = alt.Chart(data_countries).encode(
    longitude='Longitude (average):Q',
    latitude='Latitude (average):Q',
)

text = base.mark_text(dy=-5, align='right').encode(
    alt.Text('country', type='nominal'),
    opacity=alt.condition(~selector, alt.value(0), alt.value(1))
)

points = base.mark_point().encode(
    color=alt.condition(selector, 'country:N', alt.value('lightgray'), 
                        legend=alt.Legend(orient='top', columns=12, symbolLimit=50), title='Select Country'),
    size=alt.condition(~hover, alt.value(30), alt.value(100))
).add_selection(hover).add_selection(selector)


# Define the 3 Charts

# Health Charts

chart_3 = alt.Chart(data_sec_ess, width=600, height=300).mark_bar().encode(
    y=alt.Y('Name:N',
            sort=alt.SortField(field='health_risk', order='descending'), 
            axis=alt.Axis(grid=False,title='Sectors',labelLimit=100)),
    x=alt.X('health_risk:Q', axis=alt.Axis(grid=False,title='Health Risk (% workforce at risk)',format='%')),
    color = alt.Color('essential_text', scale=alt.Scale(range=['blue', 'lightblue', 'orange'], 
                                                        domain=['Essential', 'Non-Essential', 'All Sectors']),
                                                        legend=alt.Legend(title='')),
    tooltip=[alt.Tooltip('country:N'), alt.Tooltip('Name:N'),
             alt.Tooltip('health_risk:Q',format='.2%'),
             alt.Tooltip('Employment:Q',format='.2%')]
).add_selection(selector)

chart_4 = alt.Chart(data_sec_ess, width=600, height=300).mark_bar(size = 12).encode(
    x=alt.X('Name:N', 
            axis=alt.Axis(grid=False,title='Sectors',labelLimit=100), sort=alt.SortField(field='Employment', order='descending')),
    y=alt.Y('Teleworkers:Q', axis=alt.Axis(grid=False,title='Teleworkers (% of total Employment)',format='%')),
    color=alt.ColorValue('#62B1D0'))

chart_5 = alt.Chart(data_sec_ess, width=600, height=300).mark_circle().encode(
    x = alt.X('Employment:Q', axis=alt.Axis(grid=False, title='Employment (% of pre-shock level)',format='%')),
    y = alt.Y('Teleworker_sector:Q', axis=alt.Axis(grid=False,title='Workable from home (% of total Employment)',format='%')),
    size = alt.Size('emp_share:Q',legend=None),
    color = alt.Color('essential_text', scale=alt.Scale(range=['blue', 'lightblue', 'orange'], 
                                                        domain=['Essential', 'Non-Essential', 'All Sectors']),
                      legend=None),
    tooltip = [alt.Tooltip('country:N'), alt.Tooltip('Name:N'),alt.Tooltip('health_risk:Q',format='.2%'),
             alt.Tooltip('Employment:Q',format='.2%'), alt.Tooltip('Teleworker_sector:Q',format='.2%')]).add_selection(selector)

chart_6 = alt.Chart(data_sec_ess,width=600, height=300).mark_rule(strokeDash=[8,8],opacity=0.5).transform_filter(
    'datum.Name=="All Sectors"')

# Economic Charts

chart_1 = alt.Chart(data_agg, width=600, height=300).mark_bar().encode(
    y=alt.Y('label:N',sort=alt.SortField(field='Total Output', order='descending'), axis=alt.Axis(grid=False,title='Sectors Closed')),
    x=alt.X('Total Output:Q',stack=None, axis=alt.Axis(grid=False,title='Aggregate Econ. Risk (%)',format='%')),
    color = alt.condition(selector_sector, 'label:N', alt.value('lightgray'), legend=None),
    tooltip=[alt.Tooltip('country:N'), alt.Tooltip('label:N'), alt.Tooltip('Total Output:Q',format='.2%',title='GDP loss')]
).add_selection(selector).add_selection(selector_sector)

chart_2 = alt.Chart(data_sec, width=600, height=300).mark_bar().encode(
    y=alt.Y('dsName:N',sort=alt.SortField(field='Output', order='descending'), 
            axis=alt.Axis(grid=False,orient='left',title=None,labelLimit=100)),
    x=alt.X('Output:Q',stack=None,axis=alt.Axis(grid=False,title='Economic Risk (%)',format='%')),
    color=alt.condition(selector_sector, 'label:N', alt.value('lightgray')),
    opacity = alt.condition('datum.type=="closed"',alt.value(1), alt.value(.5)),
    tooltip=[alt.Tooltip('country:N'), alt.Tooltip('label:N'),
             alt.Tooltip('dsName:N'), alt.Tooltip('Output:Q',format='.2%')]
).add_selection(selector)


# Country Charts

chart_7 = alt.Chart(data_country, width=600, height=300).mark_circle().encode(
    x = alt.X('health_risk:Q', axis=alt.Axis(grid=False, title='Health Risk (% of employment)',format='%')),
    y = alt.Y('economic_risk:Q', axis=alt.Axis(grid=False,title='Economic Risk (% of GDP loss)',format='%')),
    color = alt.Color('country:N',legend=None),
    tooltip = [alt.Tooltip('country:N'),alt.Tooltip('health_risk:Q',format='.2%'),
             alt.Tooltip('economic_risk:Q',format='.2%')]).properties(title="Risks by Country")

chart_8 = alt.Chart(data_country,width=600, height=300).mark_rule(strokeDash=[8,8],opacity=0.5).transform_filter(
    'datum.country=="Mean"')


# Encode all charts

filter_chart_1 = chart_1.transform_filter(
    selector).add_selection(
    selector_sector).properties(title="Aggregate Economic Risk")

filter_chart_2 = chart_2.add_selection(selector).transform_filter(
    selector).add_selection(selector_sector).transform_filter(
    selector_sector).properties(title="Economic Risk by Sector")

filter_chart_3 = chart_3.add_selection(selector).transform_filter(
    selector)

filter_chart_4 = chart_4.transform_filter(selector)

filter_chart_5 = chart_5.transform_filter(selector).properties(title="Health risk")

filter_chart_6 = chart_6.transform_filter(selector).encode(
    x = 'Employment:Q')
filter_chart_7 = chart_6.transform_filter(selector).encode(
    y = 'Teleworker_sector:Q')

filter_chart_8 = chart_8.encode(
    x = 'health_risk:Q')
filter_chart_9 = chart_8.encode(
    y = 'economic_risk:Q')




###Sectors most exposed to health risks


In [0]:
#hide_input
((background + points + text) & (filter_chart_5 + filter_chart_6 + filter_chart_7 ) & (filter_chart_3 )).resolve_scale(color='independent')

### Sectors most exposed to economic risks

In [0]:
#hide_input
(background + points + text).resolve_scale(color='independent') & (filter_chart_1 & filter_chart_2).resolve_scale(color='shared')


###Risks by country

In [0]:
#hide_input
(chart_7 + filter_chart_8 + filter_chart_9)